## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Farsala,GR,39.2947,22.3847,48.00,81,75,6.91,broken clouds
1,1,Faanui,PF,-16.4833,-151.7500,82.27,74,73,10.80,broken clouds
2,2,Rokytnice Nad Jizerou,CZ,50.7256,15.4336,30.00,88,99,9.35,overcast clouds
3,3,Agaro,ET,7.8500,36.6500,69.03,51,22,1.39,few clouds
4,4,Butaritari,KI,3.0707,172.7902,81.77,76,57,9.37,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Faanui,PF,-16.4833,-151.7500,82.27,74,73,10.80,broken clouds
4,4,Butaritari,KI,3.0707,172.7902,81.77,76,57,9.37,light rain
7,7,Acarau,BR,-2.8856,-40.1200,77.45,87,97,9.55,overcast clouds
9,9,Mildura,AU,-34.2000,142.1500,84.20,34,9,6.91,clear sky
12,12,Kapaa,US,22.0752,-159.3190,72.00,88,90,10.36,heavy intensity rain
16,16,Rikitea,PF,-23.1203,-134.9692,79.25,75,47,19.19,scattered clouds
17,17,Atuona,PF,-9.8000,-139.0333,79.63,66,88,14.63,overcast clouds
18,18,Digboi,IN,27.3833,95.6333,75.20,64,75,4.61,haze
20,20,Vogan,TG,6.3333,1.5333,82.40,83,20,5.75,few clouds
22,22,Hobart,AU,-42.8794,147.3294,73.99,56,75,13.80,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.    
preferred_cities_df.count()  

# Assumption: There are no empty rolls, only country missing 

City_ID                247
City                   247
Country                247
Lat                    247
Lng                    247
Max Temp               247
Humidity               247
Cloudiness             247
Wind Speed             247
Current Description    247
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.  
preferred_cities_df.info()

# Assume no empty rows
clean_df = preferred_cities_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247 entries, 1 to 665
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              247 non-null    int64  
 1   City                 247 non-null    object 
 2   Country              247 non-null    object 
 3   Lat                  247 non-null    float64
 4   Lng                  247 non-null    float64
 5   Max Temp             247 non-null    float64
 6   Humidity             247 non-null    int64  
 7   Cloudiness           247 non-null    int64  
 8   Wind Speed           247 non-null    float64
 9   Current Description  247 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 21.2+ KB


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Faanui,PF,82.27,broken clouds,-16.4833,-151.7500,
4,Butaritari,KI,81.77,light rain,3.0707,172.7902,
7,Acarau,BR,77.45,overcast clouds,-2.8856,-40.1200,
9,Mildura,AU,84.20,clear sky,-34.2000,142.1500,
12,Kapaa,US,72.00,heavy intensity rain,22.0752,-159.3190,
16,Rikitea,PF,79.25,scattered clouds,-23.1203,-134.9692,
17,Atuona,PF,79.63,overcast clouds,-9.8000,-139.0333,
18,Digboi,IN,75.20,haze,27.3833,95.6333,
20,Vogan,TG,82.40,few clouds,6.3333,1.5333,
22,Hobart,AU,73.99,broken clouds,-42.8794,147.3294,


In [8]:
# slow retrieval
import time

In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"]=f"{lat},{lng}"
        
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
       
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    time.sleep(.5)
  
hotel_df.head()

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Faanui,PF,82.27,broken clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
4,Butaritari,KI,81.77,light rain,3.0707,172.7902,Isles Sunset Lodge
7,Acarau,BR,77.45,overcast clouds,-2.8856,-40.1200,castelo pizzaria
9,Mildura,AU,84.20,clear sky,-34.2000,142.1500,Mercure Hotel Mildura
12,Kapaa,US,72.00,heavy intensity rain,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach


In [10]:
# 7. Drop the rows where there is no Hotel Name.                               

clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != "")]  
#clean_hotel_df.info()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Faanui,PF,82.27,broken clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
4,Butaritari,KI,81.77,light rain,3.0707,172.7902,Isles Sunset Lodge
7,Acarau,BR,77.45,overcast clouds,-2.8856,-40.1200,castelo pizzaria
9,Mildura,AU,84.20,clear sky,-34.2000,142.1500,Mercure Hotel Mildura
12,Kapaa,US,72.00,heavy intensity rain,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Datinto a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name<dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather<dd>{Current Description}</dd> and <dt>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Faanui,PF,82.27,broken clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
4,Butaritari,KI,81.77,light rain,3.0707,172.7902,Isles Sunset Lodge
7,Acarau,BR,77.45,overcast clouds,-2.8856,-40.1200,castelo pizzaria
9,Mildura,AU,84.20,clear sky,-34.2000,142.1500,Mercure Hotel Mildura
12,Kapaa,US,72.00,heavy intensity rain,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
...,...,...,...,...,...,...,...
654,Gorontalo,ID,86.29,scattered clouds,0.5412,123.0595,Maqna Hotel by Prasanthi
658,Kati,ML,78.80,clear sky,12.7441,-8.0726,samassa la
659,Isabela,PH,82.40,heavy intensity rain,6.7085,121.9711,Isawad Resort
661,Le Port,RE,81.00,clear sky,-20.9373,55.2919,Villa Maido


In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))